In [1]:
import pandas as pd
import math
import numpy as np
import copy
import warnings
import gc
from sklearn import preprocessing
warnings.filterwarnings('ignore')
from datetime import date, timedelta
from sklearn.metrics import roc_auc_score,mean_squared_error
# from com_util import *

In [2]:
path = 'jdata/'
product_info=pd.read_csv(path+"jdata_product.csv")
user_info=pd.read_csv(path+"jdata_user.csv")
user_action=pd.read_csv(path+"jdata_action.csv")
shop_info=pd.read_csv(path+"jdata_shop.csv")
user_comment=pd.read_csv(path+"jdata_comment.csv")

In [3]:
user_action['a_date']=user_action['action_time'].apply(lambda x: str(x)[:10])
# product_info['a_date']=product_info['']
# user_info['a_date']=user_info['']

# shop_info['a_date']=shop_info['']
# user_comment['a_date']=user_comment['']

In [4]:
print(user_action.head())
print(user_info.head())
print(shop_info.head())
print(product_info.head())
print(user_comment.head())
print(user_action.shape)
print(user_info.shape)
print(shop_info.shape)
print(product_info.shape)
print(user_comment.shape)

   user_id  sku_id          action_time  module_id  type      a_date
0   937922  357022  2018-02-04 08:28:15    8107857     1  2018-02-04
1   937922      73  2018-02-04 08:27:07    8107857     1  2018-02-04
2   937922   29583  2018-02-04 08:26:31    8107857     1  2018-02-04
3   937922  108763  2018-02-04 08:26:10    8107857     1  2018-02-04
4  1369473  331139  2018-02-03 21:55:49    3712240     1  2018-02-03
   user_id  age  sex            user_reg_tm  user_lv_cd  city_level  province  \
0   381581  NaN  NaN  2018-01-21 21:03:37.0           1         NaN       NaN   
1   478401  NaN  NaN    2009-07-17 17:18:07           7         NaN       NaN   
2   581131  NaN  NaN    2012-12-10 13:02:25           7         NaN       NaN   
3  1154151  NaN  NaN  2014-02-01 16:08:00.0           5         NaN       NaN   
4  1603505  NaN  NaN    2012-03-03 15:04:36           5         NaN       NaN   

   city  county  
0   NaN     NaN  
1   NaN     NaN  
2   NaN     NaN  
3   NaN     NaN  
4   NaN  

In [5]:
product_info.cate.value_counts()

7     72420
24    24517
81    23273
75    15876
56    13265
69    12153
70    11703
27     9720
10     9185
77     8114
34     7509
62     7507
73     7137
23     6161
43     5627
12     5621
3      5144
72     5114
67     4930
20     4563
17     4395
63     4270
29     3548
37     3533
49     3381
19     3329
35     3327
55     3002
25     2976
45     2918
      ...  
39     1306
11     1249
52     1168
21     1168
26     1047
8      1014
30      855
61      794
14      786
38      782
42      576
31      552
78      548
60      515
66      483
28      468
48      449
58      402
59      373
36      349
4       315
1       283
50      200
54      179
65      136
40      111
5        57
18       42
6         8
13        1
Name: cate, Length: 81, dtype: int64

# 二分类的思路baseline

In [6]:
'''
正样本的构建：已经购买的记录作为正样本
初步想法：负样本的构建，当对一个商品一系列action后,过了两周都没有购买，将此记录记为负样本
正常来说应该时间来分析
'''

'\n正样本的构建：已经购买的记录作为正样本\n初步想法：负样本的构建，当对一个商品一系列action后,过了两周都没有购买，将此记录记为负样本\n正常来说应该时间来分析\n'

In [7]:
#已经购买的商品id
typebuy = user_action[user_action.type == 2][['user_id','sku_id']]
typebuy.shape
productbuy = typebuy.merge(user_action,on = ['user_id','sku_id'],how = 'left')
productbuy.shape

(7983755, 6)

In [8]:
#可能购买的商品id
productpotential.shape
user_action.sku_id.isin(typebuy.sku_id)

NameError: name 'productpotential' is not defined

In [ ]:
tmpbuy = productbuy.iloc[:,:].\
                groupby(['user_id','sku_id']).type.value_counts().unstack().reset_index() #.sort_values(by = 'action_time',ascending=True).\
tmpbuy.columns = ['user_id','sku_id']+['type'+str(i) for i in range(1,6)]
tmpbuy.head()

In [ ]:
tmppo = user_action.iloc[:,:].\
        groupby(['user_id','sku_id']).type.value_counts().unstack().add_prefix('type').reset_index()#sort_values(by = 'action_time',ascending=True).\
# tmppo.columns = ['user_id','sku_id']+['type'+str(i) for i in range(1,2)]
tmppo.fillna(0,inplace=True)

In [ ]:
#已经购买的记录作为正样本
possamples = productbuy
possamples.shape

In [ ]:
'''
未购买的商品记录为潜在商品记录
'''
skupo = tmppo[tmppo.type2==0]
skupo.head()
skupo = skupo[['user_id','sku_id']]
skupo.shape
productpo = skupo.merge(user_action,on = ['user_id','sku_id'],how = 'left')
productpo.shape

In [ ]:
productpo = productpo.sort_values(by = ['user_id','sku_id'],kind='mergesort')

In [ ]:
'''
找出用户对某一商品的最新一次操作的时间
#negsamples = 任一其它行为发生在2周前
'''
latestactiontime = []
from tqdm import*

for i in tqdm_notebook(range(60)):
# for i in range(60):
    print(i)
    tmp = productpo[['user_id','sku_id','a_date']].iloc[i*500000:(i+1)*500000,:].groupby(['user_id','sku_id']).\
                    apply(lambda x: max(x.a_date)<'2018-04-01').rename('outdate').reset_index()
    latestactiontime.append(tmp)
tmp = productpo[['user_id','sku_id','a_date']].iloc[30000000:,:].groupby(['user_id','sku_id']).\
                    apply(lambda x: max(x.a_date)<'2018-04-01').rename('outdate').reset_index()
latestactiontime = pd.concat(latestactiontime, axis = 0)

In [ ]:
latestactiontime.shape

In [ ]:
tmp = latestactiontime[latestactiontime.outdate]
negsamples = tmp.merge(productpo,on = ['user_id','sku_id'],how = 'left')

In [ ]:
negsamples.drop(['outdate'],axis = 1,inplace=True)
negsamples.head()

In [ ]:
'''
潜在商品里取出了负样本，从剩下的记录里构建测试集数据
初步思路：从最近一次操作的时间在一周内的记录作为测试集
'''
tmp = latestactiontime[~latestactiontime.outdate]
testsamples = tmp.merge(productpo,on = ['user_id','sku_id'],how = 'left')
testsamples.drop(['outdate'],axis = 1,inplace=True)
testsamples.shape

In [ ]:
targetday = '2018-04-16'
testsamples = testsamples.sort_values(by = ['user_id','sku_id'],kind='mergesort')

In [ ]:
lastweek = testsamples[['user_id','sku_id','a_date']].groupby(['user_id','sku_id']).\
                    apply(lambda x: max(x.a_date)>'2018-04-08').rename('lastweek').reset_index()
tmp = lastweek[lastweek.lastweek]
testsamples = tmp.merge(testsamples,on = ['user_id','sku_id'],how = 'left')
testsamples.drop(['lastweek'],axis = 1,inplace=True)
testsamples.shape

# data_constuct

In [ ]:
'''
训练集构建
'''
possamples['label'] = 1
negsamples['label'] = 0
trainraw = pd.concat([possamples,negsamples],axis = 0)

In [ ]:
'''
将用户表和商品表连接，各自的信息可以作为初步的特征
'''
trainraw = trainraw.merge(user_info,on = 'user_id',how = 'left')
trainraw = trainraw.merge(product_info,on = 'sku_id',how = 'left')

In [ ]:
'''
数据量过大，在连接的时候要做内存优化
初步：去掉object类型的列，将数值型的列用‘int’c存放在df中

'''
cols = [u'action_time', u'module_id','a_date',
      u'user_reg_tm', u'market_time']
trainraw = trainraw.drop(cols, axis= 1).fillna(0).astype(int)

In [ ]:
trainraw.info()

In [ ]:
trainraw = trainraw.merge(shop_info,on = 'shop_id',how = 'left')


In [ ]:
cols = [u'shop_reg_tm', u'shop_id']
trainraw = trainraw.drop(cols, axis= 1).fillna(0).astype(int)

In [ ]:
trainraw.head()

In [ ]:
'''
数据量过大，在连接的时候要做内存优化
初步：去掉object类型的列，将数值型的列用‘int16’c存放在df中

'''
#内存优化
cols = [u'type', u'label', u'age', u'sex', u'user_lv_cd',
        u'city_level', u'province', u'city', u'county', u'brand', u'cate_x',
        u'cate_y', u'shop_score']
trainraw[cols] = trainraw[cols].astype('int16')

#对测试集做连接

In [ ]:
testraw = testsamples.copy()

In [ ]:
testraw = testraw.merge(user_info,on = 'user_id',how = 'left')
testraw = testraw.merge(product_info,on = 'sku_id',how = 'left')
cols = [u'action_time', u'module_id','a_date',
      u'user_reg_tm', u'market_time']
testraw = testraw.drop(cols, axis= 1).fillna(0).astype(int)

In [ ]:
testraw = testraw.merge(shop_info,on = 'shop_id',how = 'left')

In [ ]:
cols = [u'shop_reg_tm', u'shop_id']
testraw = testraw.drop(cols, axis= 1).fillna(0).astype(int)

In [ ]:
'''
数据量过大，在连接的时候要做内存优化
初步：去掉object类型的列，将数值型的列用‘int16’c存放在df中

'''
#内存优化
cols = [u'type', u'age', u'sex', u'user_lv_cd',
        u'city_level', u'province', u'city', u'county', u'brand', u'cate_x',
        u'cate_y', u'shop_score']
testraw[cols] = testraw[cols].astype('int16')

# fea_construct

In [ ]:
'''
用户对商品的历史行为记录，可以统计来做特征
正常来说，应该结合时间顺序来考虑
'''
tmp = trainraw.groupby(['user_id','sku_id']).type.value_counts().unstack().add_prefix('type').fillna(0).astype('int16').reset_index()

In [ ]:
tmp.fillna(0,inplace=True)
traindata = trainraw.drop_duplicates(subset = ['user_id','sku_id'],keep='first')
traindata = traindata.sort_values(by = ['user_id','sku_id'],kind='mergesort')
tmp = tmp.sort_values(by = ['user_id','sku_id'],kind='mergesort')
traindata = traindata.merge(tmp,on = ['user_id','sku_id'],how = 'left')

In [ ]:
tmp = testraw.groupby(['user_id','sku_id']).type.value_counts().unstack().add_prefix('type').fillna(0).astype('int16').reset_index()
# tmp.fillna(0,inplace=True)
testdata = testraw.drop_duplicates(subset = ['user_id','sku_id'],keep='first')
testdata = testdata.sort_values(by = ['user_id','sku_id'],kind='mergesort')
tmp = tmp.sort_values(by = ['user_id','sku_id'],kind='mergesort')
testdata = testdata.merge(tmp,on = ['user_id','sku_id'],how = 'left')

In [ ]:
print(traindata.info())
testdata.info()

In [ ]:
traindata.drop('type2',axis =1,inplace =True)

In [ ]:
X_train = traindata[testdata.columns]
X_test = testdata

In [ ]:
Y_train = traindata['label']

In [ ]:
''' 在做二分类的时候，样本不平衡，应当解决，降采样'''
#降采样
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0, replacement=True)
# X_resampled, y_resampled = rus.fit_sample(X_train, Y_train)

# models

In [ ]:
'''初步尝试lgb作为分类器'''
#lgb

#数据集划分时在索引上容易出错，先对数据冲索引
X_train = X_train.reset_index().drop('index',axis=1)
Y_train = Y_train.reset_index().drop('index',axis=1)
print(X_train.shape)

In [ ]:
#数据集划分时在索引上容易出错，先对数据冲索引
X,x_no,Y,y_no = train_test_split(X_train,Y_train,train_size = 0.2,random_state = 2019)
X = X.reset_index().drop('index',axis=1)
Y = Y.reset_index().label

In [ ]:
#官方评价指标
def f11(y_true1,y_pred1):
    r = recall_score(y_true=y_true1,y_pred=y_pred1)
    p = precision_score(y_true=y_true1,y_pred=y_pred1)
    return (3*r*p) / (2*r+p)

def f12(y_true2,y_pred2):
    r = recall_score(y_true=y_true2,y_pred=y_pred2)
    p = precision_score(y_true=y_true2,y_pred=y_pred2)
    return (5*r*p) / (2*r+3*p)
def eval(f11,f12):
    return 0.4*f11+0.6*f12

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit,KFold,train_test_split
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
seed = 2019
n_splits=5
kfold =  StratifiedShuffleSplit(n_splits=5, random_state = seed)
prediction = 0
'''
实际上训练集和验证集应该按时间分开划分
'''
for idx,(trainindex,testindex) in enumerate(kfold.split(X,Y)):
    x_train,x_test,y_train,y_test = X.iloc[trainindex],X.iloc[testindex],Y[trainindex],Y[testindex]
    
    x_tr = lgb.Dataset(x_train,label=y_train)
    x_te = lgb.Dataset(x_test,label=y_test)
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',  # 设置提升类型
        'objective': 'binary', # 目标函数
        'metric': {'l2', 'auc'},  # 评估函数
        'num_leaves': 31,   # 叶子节点数
        'learning_rate': 0.05,  # 学习速率
        'feature_fraction': 0.9, # 建树的特征选择比例
        'bagging_fraction': 0.9, # 建树的样本采样比例
        'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
        'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
    }

    bst = lgb.train(params, x_tr,10,valid_sets = [x_te] ,early_stopping_rounds=10,verbose_eval=20) 
    pred = bst.predict(x_test)
    print(accuracy_score(y_test,pred>0.9))
    
    #5折平均作为最终的预测概率
    prediction += bst.predict(X_test)
prediction = prediction / n_splits

In [ ]:
np.sum(prediction>0.5)

In [ ]:
#时序关联分析

# postprocessing

In [ ]:
tmp = X_test[prediction>0.5][['user_id','sku_id']]
result = tmp.merge(product_info,on = 'sku_id',how = 'left')

In [ ]:
result = result.drop(['brand','sku_id','market_time'],axis = 1)

In [ ]:
result = result.drop_duplicates().dropna().astype(int)
result.head()

# submission

In [ ]:
result = result[['user_id','cate','shop_id']]
result.to_csv('../output/result'+ str(datetime.now().date())+'.csv',index = False,)

# 其他分析

# shop prediction

In [ ]:
samples = user_action[user_action['a_date']<='2018-4-15'].merge(product_info,on = 'sku_id',how = 'left')


In [ ]:
import time
import datetime
date_wins = [
#              ('2018-04-09','2018-04-15'),
#              ('2018-04-02','2018-04-08'),
#              ('2018-03-26','2018-04-01'),
#              ('2018-03-19','2018-03-25'),
#              ('2018-03-12','2018-03-18'),
#              ('2018-03-05','2018-03-11'),
#              ('2018-02-28','2018-03-04'),
#              ('2018-02-21','2018-02-27'),
#              ('2018-02-14','2018-02-20'),
#              ('2018-02-07','2018-02-13'),
#              ('2018-02-01','2018-02-06')
            ]
for date in date_wins:
    start = datetime.datetime.now()
    print('counting...'+date[0] + ' to '+ date[1])
    tmp = samples[(samples['a_date']>=date[0])&(samples['a_date']<=date[1])].head(500000).groupby('user_id').shop_id.value_counts().unstack()
    tmp.columns = ['shop'+str(int(i)) for i in tmp.columns]
    tmp.fillna(0,inplace=True)
    print('saving...'+date[0] + ' to ' + date[1])
    print(tmp.shape)
    tmp.to_csv('../data/user_shopmatrix'+date[0]+'.csv',index=False) 
    end = datetime.datetime.now()
    print(end-start)
# actionmatrix = samples.iloc[:3000000,:].groupby('user_id').shop_id.value_counts().unstack()

In [ ]:
import time
import datetime
date_wins = [
             ('2018-04-09','2018-04-15'),
             ('2018-04-02','2018-04-08'),
             ('2018-03-26','2018-04-01'),
             ('2018-03-19','2018-03-25'),
             ('2018-03-12','2018-03-18'),
             ('2018-03-05','2018-03-11'),
             ('2018-02-28','2018-03-04'),
             ('2018-02-21','2018-02-27'),
             ('2018-02-14','2018-02-20'),
             ('2018-02-07','2018-02-13'),
             ('2018-02-01','2018-02-06')
            ]
for date in date_wins:
    start = datetime.datetime.now()
    print('counting...'+date[0] + ' to '+ date[1])
    tmp = samples[(samples['a_date']>=date[0])&(samples['a_date']<=date[1])].head(500000).groupby('user_id').cate.value_counts().unstack()
    tmp.columns = ['shop'+str(int(i)) for i in tmp.columns]
    tmp.fillna(0,inplace=True)
    print('saving...'+date[0] + ' to ' + date[1])
    print(tmp.shape)
    tmp.to_csv('../data/user_catematrix'+date[0]+'.csv',index=False) 
    end = datetime.datetime.now()
    print(end-start)
# actionmatrix = samples.iloc[:3000000,:].groupby('user_id').shop_id.value_counts().unstack()

In [ ]:
del tmp

In [ ]:
actionmatrix = pd.read_csv('../data/user_catematrix2018-04-09.csv')

# 马尔可夫链/CRF解决时序问题

In [ ]:
catelabel=label.iloc[:,:].groupby(['user_id','a_date'])["cate"].unique().reset_index()

In [ ]:
tmp[:5]

In [ ]:
catelabel.head()

In [ ]:
train.head()

In [ ]:
train_sample_list